In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Exercise 08 - Monte Carlo simulation

#### 8.1) Lennard-Jones Potential

#### 8.2) 2D-potential

First we show the potential landscape and the corresponding distribution. Afterwards we will answers the questions.

For this problem, since the units of the potential were not given, we assume that it has units of $k_{B}T$. For the final question, we will consider different temperature values by varying a coefficient.

In [ ]:
def U(x, y):
    return np.square(x**2 - 1) + (x - y) + y**2

def uu(xx):
    """Wrapper for an np.ndarray. xx[..., 0] = x, xx[..., 1] = y"""
    return U(xx[..., 0], xx[..., 1])

In [ ]:
endpt = 2.5
numpt = 100

xx = np.linspace(-endpt, endpt, numpt)
yy = np.linspace(-endpt, endpt, numpt)
x, y = np.meshgrid(xx, yy)
z = U(x, y)

fig, ax = plt.subplots()
ax.contourf(x, y, z, levels=20)
ax.set_title("Contour plot of Energy Landscape")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

In [ ]:
def Boltzmann(x, y, beta=1.0):
    return np.exp(-beta * U(x, y))

def boltzmann(xx, beta=1.0):
    """Wrapper for an np.ndarray. xx[..., 0] = x, xx[..., 1] = y"""
    return Boltzmann(xx[..., 0], xx[..., 1], beta=1.0)

In [ ]:
z = Boltzmann(x, y, beta=1.0)

fig, ax = plt.subplots()
ax.contourf(x, y, z, levels=20)
ax.set_title("Boltzmann Distribution")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

Metropolis MCMC for sampling.

In [ ]:
def metropolis(state, step_size=1.0, beta=1.0):
    step = (2 * np.random.rand(*state.shape) - 1) * step_size
    proposal = state + step
    diff = uu(proposal) - uu(state)
    if diff <= 0 or (np.random.rand() < np.exp(-beta * diff)):
        return proposal
    else:
        return state
    
def mcmc(initial_state, steps, step_size=1.0, beta=1.0):
    states = [initial_state]
    for _ in range(steps):
        state = metropolis(states[-1], step_size=step_size, beta=beta)
        states.append(state)
    return np.asarray(states)

In [ ]:
states = mcmc(np.array([0.0, 0.0]), 10000)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(*states.T)
ax.set_title("Samples from MCMC")
ax.set_xlabel("x")
ax.set_ylabel("y")
plt.show()

Step size and effect on acceptance ratio. As step size decreases, we accept more proposals; however, we do not get good "mixing." Mixing colloquially means visiting a significant portion of the distribution with high regularity.

In [ ]:
def acceptance(samples):
    accepted = []
    for i in range(len(samples)):
        try:
            if np.all(np.equal(samples[i], samples[i+1])):
                accepted.append(False)
            else:
                accepted.append(True)
        except IndexError:
            break
    return accepted

def acceptance_ratio(samples):
    arr = acceptance(samples)
    return sum(arr)/len(arr)

In [ ]:
ar = []
delta = np.linspace(0.01, 0.5, 25)
for d in delta:
    smps = mcmc(np.array([0.0, 0.0]), 10000, step_size=d)
    ar.append(acceptance_ratio(smps))

In [ ]:
fig, ax = plt.subplots()
ax.plot(delta, ar)
ax.set_title("Acceptance Ratio versus Step Size")
ax.set_xlabel("Delta")
ax.set_ylabel("Acceptance Ratio")
plt.show()